This notebook seeks to explore whether a portfolio's total ROI can be improved by holding a certain amount of cash reserves to 'buy the dip'--attempting to time the market by buying at relative low points rather than holding for a longer duration.

## 0. Import Modules and Data

In [1]:
import pandas as pd
import numpy as np
from pandas_datareader import data
import pandas_datareader.data as web
import datetime as dt
import Portfolio as port
import random

## 1. Set-up Portfolio

In [2]:
# Define start date and end date

start_date = '2015-01-01'

end_date = '2020-01-05'

### 1.1 Establish random time horizons

The goal here is to simulate a random timeframe of investment to evaluate if an investment strategy is consistently successful or if it is just lucky timing. The 'time_horizons' function is designed to randomly sample a range of dates (between a start date and and end date) and return a pair where the start date is earlier than the end date and the time between the two dates is greater than the specified time horizon. 

In [3]:
def time_horizons(samples, time_horizon, start_date, end_date):
    # samples reflects how many date pairs to return
    # The time_horizon is how long the dates need to be apart from eachother
    # start_date and end_date refer to the absolute max and min of the dates samples
    time_period = pd.date_range(pd.to_datetime(start_date),pd.to_datetime(end_date))# Full timeline for data as a range
    counter = 0
    date_dict = {}
    while counter < samples:
        random_start = random.choice(time_period)
        random_end = random.choice(time_period)
        while random_start > random_end or (random_end-random_start) < dt.timedelta(days=365*time_horizon):
            random_start = random.choice(time_period)
            random_end = random.choice(time_period)
        date_dict[random_start] = random_end
        counter += 1
    return(date_dict)

In [4]:
# Create a dictionary of date pairs to test the investing strategy.

dates = time_horizons(20,3,pd.to_datetime('01/01/2010'), pd.to_datetime('12/31/2020'))

In [5]:
# Check the date pairs to ensure parameters are met
for item in dates.items():
    print(item)

(Timestamp('2013-09-03 00:00:00', freq='D'), Timestamp('2019-08-18 00:00:00', freq='D'))
(Timestamp('2015-01-23 00:00:00', freq='D'), Timestamp('2020-12-05 00:00:00', freq='D'))
(Timestamp('2010-12-03 00:00:00', freq='D'), Timestamp('2019-01-30 00:00:00', freq='D'))
(Timestamp('2013-03-29 00:00:00', freq='D'), Timestamp('2020-10-03 00:00:00', freq='D'))
(Timestamp('2010-05-26 00:00:00', freq='D'), Timestamp('2019-01-01 00:00:00', freq='D'))
(Timestamp('2010-01-06 00:00:00', freq='D'), Timestamp('2020-10-11 00:00:00', freq='D'))
(Timestamp('2012-03-27 00:00:00', freq='D'), Timestamp('2016-02-09 00:00:00', freq='D'))
(Timestamp('2016-07-07 00:00:00', freq='D'), Timestamp('2020-06-21 00:00:00', freq='D'))
(Timestamp('2014-08-24 00:00:00', freq='D'), Timestamp('2019-12-13 00:00:00', freq='D'))
(Timestamp('2010-01-13 00:00:00', freq='D'), Timestamp('2013-04-18 00:00:00', freq='D'))
(Timestamp('2012-04-07 00:00:00', freq='D'), Timestamp('2020-02-11 00:00:00', freq='D'))
(Timestamp('2012-10-0

### 1.2 Create a Master Log of all portfolio outcomes

In [6]:
### Create empty dataframe to be populated during the run

master_log = pd.DataFrame(columns = ['Portfolio','Strategy','Start Date','End Date','Time Horizon', 'ROI'])

In [7]:
master_log

,Portfolio,Strategy,Start Date,End Date,Time Horizon,ROI


### 1.3 Set Up Strategies for Comparison

In [8]:
# Define Shares

# Provide the name of the ticker and type (Equity or Bond)

#VOO Vanguard ETF is serving as a proxy for individual stocks/bonds

voo = port.Share('VOO', 'Equity', start_date,end_date)

In [9]:
shares_list = [voo]

shares_dict = {}

for share in shares_list:
    shares_dict[share] = share.type

In [10]:
# Define Strategy

# Provide the equity distribution, the bond distriubtion, cash distribution, and the threshold
strat = port.Strategy(100,0,0,1)
# For this strategy of trying to time the market I set up the portfolio with a 50/50 allocation of equities and cash
# This is done so that the portfolio can have enough cash to buy any big dips early on.

In [11]:
# Actions measured in days, not minutes...

# the more the share value drops, the more cash we put in --wait until the panic has subsided

In [12]:
start_date

'2015-01-01'

## 2. Run Strategies

In [13]:
### 2.0 Set Parameters

In [14]:
# Set number of samples

samples = 20

In [15]:
# Set time horizon (years expecting to invest)

horizon = 3

### 2.1 Strategy 1: Buy & Hold

In [16]:
# Generate date pairs

dates = time_horizons(samples,horizon,pd.to_datetime('01/01/2015'), pd.to_datetime('12/31/2020'))

In [17]:
start_date

'2015-01-01'

In [19]:
# Run without trying to time the market
for start_date in dates:
    end_date = dates[start_date]
    time_period = pd.date_range(pd.to_datetime(start_date),pd.to_datetime(end_date))
    portfolio = port.Portfolio(shares_dict, div_reinvest = True)
    portfolio.initial_buy(500, strat, pd.to_datetime(start_date))
    # set a counter to avoid buying into shares more than once a week
    counter = 0
    for day in time_period:
        counter +=1
        portfolio.reinvest_divs(day)
        portfolio.get_asset_values(day)
    #    for share in portfolio.shares:
    #         share.get_rolling(day)
    #         last_week = day - dt.timedelta(days = 7)
    #         prev_week = last_week - dt.timedelta(days = 7)
    #         week_avg = share.rolling_90_days[share.rolling_90_days.index>=last_week]['Low'].mean()
    #         ninety_day_avg = share.rolling_90_days.Low.mean()
    #         if (week_avg-ninety_day_avg)/ninety_day_avg < -0.05 and counter >=7:
    #             buy_amount = (((abs((week_avg-ninety_day_avg)/share.rolling_90_days.Low.mean())+1)**2)-1)*portfolio.cash_bal
    #             portfolio.buy(share, buy_amount, day)
    #             counter = 0
    portfolio_history = portfolio.get_hist_df()
    portfolio_history['Date'] = pd.to_datetime(portfolio_history['Date']) # Convert date to datetime object
    portfolio_history.set_index('Date', inplace = True)
    roi = portfolio_history.loc[end_date,'Total Value']/portfolio_history.loc[start_date,'Total Value']
    master_log = master_log.append({'Portfolio':'Portfolio 1','Strategy':'Buy & Hold','Start Date':start_date,'End Date':end_date,'Time Horizon':horizon,'ROI':roi}, ignore_index=True)

1.4541273639636383
1.753701787609191
1.6830823546843583
1.5665180640127665
1.615111505593701
1.5013110956558728
1.7054726798576993
1.396392522980929
1.2291591162872717
1.7961229294562797
1.3788000492988306
1.6622894078763075
1.4279791702059037
1.686183369570748
1.5961396257822051
1.710908248348145
1.822658071319945
1.6997548860002951
1.7285161890810483
1.687084060506575


In [20]:
master_log

,Portfolio,Strategy,Start Date,End Date,Time Horizon,ROI
0,Portfolio 1,Buy & Hold,2015-07-17,2019-05-17,3,1.454127
1,Portfolio 1,Buy & Hold,2016-03-09,2020-10-01,3,1.753702
2,Portfolio 1,Buy & Hold,2015-11-23,2020-09-09,3,1.683082
3,Portfolio 1,Buy & Hold,2016-12-02,2020-12-23,3,1.566518
4,Portfolio 1,Buy & Hold,2016-11-05,2020-07-01,3,1.615112
5,Portfolio 1,Buy & Hold,2015-06-19,2019-08-31,3,1.501311
6,Portfolio 1,Buy & Hold,2016-04-09,2020-02-23,3,1.705473
7,Portfolio 1,Buy & Hold,2015-10-03,2018-12-14,3,1.396393
8,Portfolio 1,Buy & Hold,2015-03-25,2018-12-24,3,1.229159
9,Portfolio 1,Buy & Hold,2016-02-21,2020-04-07,3,1.796123


In [ ]:
# Explain the logic used to 'buy the dip' here:

# If the weekly average of the share value decreases by more than 5% from the 90% average
# AND
# If we haven't bought any more shares in the last 7 days...

# Then buy an amount equal to:
# The absolute value of the decrease, plus 1, squared, minus 1, times the amount of cash held by the portfolio.

# For example:
# If we have a decrease of 5% and $10 of cash it would look like this:
#

In [ ]:
(((0.05 + 1)**2)-1)*10

In [ ]:
# for i in np.arange(0.05,1.05,0.05):
#     print(i,(((i + 1)**(2+i))-1)*100) # I kinda like this formula, but we can tinker with this later...

In [ ]:
portfolio = port.Portfolio(shares_dict, div_reinvest = True)

portfolio.initial_buy(500, strat, start_date)

In [ ]:

time_period = pd.date_range(pd.to_datetime(start_date),pd.to_datetime(end_date))

# set a counter to avoid buying into shares more than once a week
counter = 0
for day in time_period:
    counter +=1
    portfolio.reinvest_divs(day)
    portfolio.get_asset_values(day)
    for share in portfolio.shares:
        share.get_rolling(day)
        last_week = day - dt.timedelta(days = 7)
        prev_week = last_week - dt.timedelta(days = 7)
        week_avg = share.rolling_90_days[share.rolling_90_days.index>=last_week]['Low'].mean()
        ninety_day_avg = share.rolling_90_days.Low.mean()
        if (week_avg-ninety_day_avg)/ninety_day_avg < -0.05 and counter >=7:
            buy_amount = (((abs((week_avg-ninety_day_avg)/share.rolling_90_days.Low.mean())+1)**2)-1)*portfolio.cash_bal
            if buy_amount > portfolio.cash_bal:
                buy_amount = portfolio.cash_bal
            portfolio.buy(share, buy_amount, day)
            counter = 0
portfolio_history = portfolio.get_hist_df()
portfolio_history['Date'] = pd.to_datetime(portfolio_history['Date']) # Convert date to datetime object
portfolio_history.set_index('Date', inplace = True)           

In [ ]:
portfolio_history

In [ ]:
voo.get_data()

In [ ]:
voo.get_rolling(start_date)

In [ ]:
voo.rolling_90_days.Low.mean()

In [ ]:
# Create a loop for different cash allocations

# This is largely the same as above, however, the strat (strategy) will change in each iteration

ini_amount = 500
 
for i in range(0,105,5):
    strat = port.Strategy(i,0,100-i,1) 
    
    portfolio = port.Portfolio(shares_dict)

    portfolio.initial_buy(ini_amount, strat, start_date)
    
    time_period = pd.date_range(pd.to_datetime(start_date),pd.to_datetime(end_date))

    for day in time_period:
        portfolio.reinvest_divs(day)
        portfolio.get_asset_values(day)
        if portfolio.asset_split['Equities'] > strat.equity_distribution+strat.threshold:
            sell_amt = (portfolio.asset_values['Equities']+portfolio.asset_values['Bonds'])*((portfolio.asset_split['Equities']-strat.equity_distribution)/100)
            sell_amt_per = sell_amt/len(portfolio.equities)
            for share in portfolio.equities: # sell equities and buy more bonds
                portfolio.sell(share, sell_amt_per, day)
            for share in portfolio.bonds:
                portfolio.buy(share, sell_amt_per, day)

        if portfolio.asset_split['Bonds'] > strat.bond_distribution+strat.threshold:
            sell_amt = (portfolio.asset_values['Equities']+portfolio.asset_values['Bonds'])*(portfolio.asset_split['Bonds']-strat.bond_distribution)
            sell_amt_per = sell_amt/len(portfolio.bonds)
            for share in portfolio.bonds: # sell bonds and buy more equities
                portfolio.sell(share, sell_amt_per, day)
            for share in portfolio.bonds:
                portfolio.buy(share, sell_amt_per, day)
    portfolio_history = portfolio.get_hist_df()
    portfolio_history['Date'] = pd.to_datetime(portfolio_history['Date']) # Convert date to datetime object
    portfolio_history.set_index('Date', inplace = True) 
    
    # Create a new column of the allocation dataframe based on the total value of the most recent portfolio simulation
    allocation_df[str(i)+'% Equities:'+str(100-i)+'% Bonds'] = portfolio_history['Total Value']
    
    # Loop through each year to calculate a ROI in each year 
    
    portfolio_history['Year'] = portfolio_history.index.strftime('%Y')
    
    years = list(portfolio_history['Year'].unique())
    
    for year in years:
        subset = portfolio_history[portfolio_history['Year']==year]
        allocation_roi_df.at[year,str(i)+'% Equities:'+str(100-i)+'% Bonds'] = ((subset['Total Value'].iat[-1]/subset['Total Value'].iat[0])-1)*100
    allocation_roi_df.at['Period Total',str(i)+'% Equities:'+str(100-i)+'% Bonds'] = ((portfolio_history['Total Value'].iat[-1]/portfolio_history['Total Value'].iat[0])-1)*100